In [1]:
import pandas as pd
import numpy as np

from requests import get
from bs4 import BeautifulSoup
import os

import acquire
import prepare

In [2]:
df = acquire.scrape_github_data()

MissingSchema: Invalid URL '': No schema supplied. Perhaps you meant http://?

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
df = prepare.prep_repo_data(df)

In [ ]:
df

In [ ]:
df.clean_stemmed.str.len()

In [ ]:
url = 'https://github.com/search?o=desc&p=1&q=petroleum&s=stars&type=Repositories'
headers = {'User-Agent': 'manual search'} 
response = get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
def get_repo_names(url):
    headers = {'User-Agent': 'manual search'} 
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    repos = [soup.find_all('a', class_='v-align-middle')[i].text\
     for i in range(10)]
    return repos

In [ ]:
# [soup.find_all('a', class_='v-align-middle')[i].text\
#  for i in range(10)]

In [ ]:
# urls = ['https://github.com/search?o=desc&p='+str(i)\
#     +'&q=petroleum&s=stars&type=Repositories' for i in range(1,11)]
# urls

In [ ]:
# test = []
# urls = ['https://github.com/search?o=desc&p=1&q=petroleum&s=stars&type=Repositories',
#  'https://github.com/search?o=desc&p=2&q=petroleum&s=stars&type=Repositories',
#  'https://github.com/search?o=desc&p=3&q=petroleum&s=stars&type=Repositories',
#  'https://github.com/search?o=desc&p=4&q=petroleum&s=stars&type=Repositories',
#  'https://github.com/search?o=desc&p=5&q=petroleum&s=stars&type=Repositories',
#  'https://github.com/search?o=desc&p=6&q=petroleum&s=stars&type=Repositories',
#  ]
# for url in urls:
#     test.append(get_repo_names(url))
# test

In [ ]:
urls = ['https://github.com/search?o=desc&p='+str(i)\
    +'&q=petroleum&s=stars&type=Repositories' for i in range(1,12)]
urls

In [ ]:
len(urls)

In [ ]:
test = []
type(test)

In [ ]:
test = []
for i, url in enumerate(urls):
    try:
        test.append(get_repo_names(url))
    except IndexError:
        break
test

In [ ]:
type(test[0][0])

In [ ]:
df = pd.DataFrame(test)

In [ ]:
df

In [ ]:
sum(test, [])